In [1]:
using Gmsh: gmsh
using Gridap
using Gridap.Fields
using Gridap.CellData
using Gridap.TensorValues
using Gridap.ReferenceFEs
using Gridap.Geometry
using GridapGmsh
using SparseArrays
using MinFEM
using WriteVTK

In [2]:
σ₀ = 1e3
h = 5e-2
hf = h/10
Lₚ = 2.0
Hₚ = 8.0
rc = 0.25

0.25

In [3]:
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.geo.addPoint(0.0, 0.0, 0.0, h, 1)
gmsh.model.geo.addPoint(Lₚ, 0.0, 0.0, h, 2)
gmsh.model.geo.addPoint(Lₚ, Hₚ, 0.0, h, 3)
gmsh.model.geo.addPoint(0, Hₚ, 0.0, h, 4)
gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 1, 4)
gmsh.model.geo.addCurveLoop([3,4,1,2],1)
gmsh.model.addPhysicalGroup(2, [1],1)

gmsh.model.geo.addPoint(Lₚ/2-rc, Hₚ/2, 0.0, hf, 5)
gmsh.model.geo.addPoint(Lₚ/2+rc, Hₚ/2, 0.0, hf, 6)
gmsh.model.geo.addPoint(Lₚ/2, Hₚ/2-rc, 0.0, hf, 7)
gmsh.model.geo.addPoint(Lₚ/2, Hₚ/2+rc, 0.0, hf, 8)
gmsh.model.geo.addPoint(Lₚ/2, Hₚ/2, 0.0, hf, 9)
gmsh.model.geo.addCircleArc(5, 9, 6, 5)
gmsh.model.geo.addCircleArc(6, 9, 5, 6)
gmsh.model.geo.addCurveLoop([5,6],2)
gmsh.model.geo.addPlaneSurface([2,-1], 1)

gmsh.model.addPhysicalGroup(1, [1],1)
gmsh.model.addPhysicalGroup(1, [3],2)
gmsh.model.addPhysicalGroup(1, [1],3)
gmsh.model.addPhysicalGroup(1, [3],4)

gmsh.model.setPhysicalName(2, 1, "PlateDomain")

gmsh.model.setPhysicalName(1, 1, "DirichletBot")
gmsh.model.setPhysicalName(1, 2, "NeumannTop") ,
gmsh.model.setPhysicalName(1, 3, "ElectricPotentialBot")
gmsh.model.setPhysicalName(1, 4, "ElectricPotentialTop")

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("PlateWithHole.msh")
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 50%] Meshing curve 4 (Line)
Info    : [ 70%] Meshing curve 5 (Circle)
Info    : [ 90%] Meshing curve 6 (Circle)
Info    : Done meshing 1D (Wall 0.00153456s, CPU 0.001396s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.402703s, CPU 0.402399s)
Info    : 14927 nodes 29857 elements
Info    : Writing 'PlateWithHole.msh'...
Info    : Done writing 'PlateWithHole.msh'


In [4]:
model = GmshDiscreteModel("PlateWithHole.msh")
writevtk(model,"PlateWithHole")

Info    : Reading 'PlateWithHole.msh'...
Info    : 16 entities
Info    : 14924 nodes
Info    : 29212 elements
Info    : Done reading 'PlateWithHole.msh'


3-element Vector{Vector{String}}:
 ["PlateWithHole_0.vtu"]
 ["PlateWithHole_1.vtu"]
 ["PlateWithHole_2.vtu"]

### Extracting Cell Points

In [5]:
order = 1
degree = 2*order
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

Measure()

## Input paramters for Piezo-Electric Material Properties

#### Input elasticity parameters

In [6]:
const C₁₁_mat = 5554.858895326576
const C₁₂_mat = 2145.4504024353614
const C₃₃_mat = 4394.907633994244

4394.907633994244

#### Input piezo-electric parameters

In [7]:
const d₁₁_mat = 0.2573464527796283
const d₂₁_mat = 17.11343246048201
const d₂₂_mat = 1.7586404723250948
const d₂₃_mat = -30.321530103414663

-30.321530103414663

#### Input permitivity parameters

In [8]:
const a₁₁_mat = 101.73193916663583
const a₂₂_mat = 97.03122816874334

97.03122816874334

## Constitutive Matrices

### Elastic stiffness tensor

In [9]:
function ElasFourthOrderConstTensor(C₁₁,C₁₂,C₃₃)
      C1111 = C₁₁
      C1122 = C₁₂
      C1112 = 0.0
      C2222 = C₁₁
      C2212 = 0.0
      C1212 = C₃₃    
      C_ten = SymFourthOrderTensorValue(C1111,C1112,C1122,C1112,C1212,C2212,C1122,C2212,C2222)
    return  C_ten
end
const C_mat = ElasFourthOrderConstTensor(C₁₁_mat,C₁₂_mat,C₃₃_mat)

SymFourthOrderTensorValue{2, Float64, 9}(5554.858895326576, 0.0, 2145.4504024353614, 0.0, 4394.907633994244, 0.0, 2145.4504024353614, 0.0, 5554.858895326576)

### Third order piezoelectric tensor

In [10]:
function PiezoThirdOrderConstTensor(d₁₁,d₂₁,d₂₂,d₂₃)
    # 1 for Plane Stress and 2 Plane Strain Condition 
      e111 = d₁₁
      e112 = 0.0
      e121 = 0.0
      e122 = 0.0
      e211 = d₂₁
      e212 = d₂₃
      e221 = d₂₃
      e222 = d₂₂   
    vals = zeros(2,2,2);
    vals[1,:,:] .= [e111 e112
                    e121 e122]
    vals[2,:,:] .= [e211 e212
                    e221 e222]
    e_ten = ThirdOrderTensorValue(vals ...)
    return  e_ten
end
const d_mat = PiezoThirdOrderConstTensor(d₁₁_mat,d₂₁_mat,d₂₂_mat,d₂₃_mat)

ThirdOrderTensorValue{2, 2, 2, Float64, 8}(0.2573464527796283, 17.11343246048201, 0.0, -30.321530103414663, 0.0, -30.321530103414663, 0.0, 1.7586404723250948)

In [11]:
const a_mat = TensorValue(a₁₁_mat,0.0,0.0, a₂₂_mat)

TensorValue{2, 2, Float64, 4}(101.73193916663583, 0.0, 0.0, 97.03122816874334)

## Stress
$\sigma_{elas}(\epsilon(\boldsymbol{u})) =  \mathbb{C}\,\boldsymbol{\epsilon}$

$\sigma_{piezo}(\nabla\phi) =  \boldsymbol{d}^T\,\boldsymbol{\nabla}\phi$

In [12]:
σ_elas(ε) = C_mat ⊙ ε

σ_piezo(∇) = ∇ ⋅ d_mat

σ_piezo (generic function with 1 method)

## Electric Displacement
$\boldsymbol{D}_{elas}(\epsilon(\boldsymbol{u})) =  \boldsymbol{d}\boldsymbol{\epsilon}$

$\boldsymbol{D}_{piezo}(\nabla\phi)  = \boldsymbol{K}\boldsymbol{\nabla} \phi$

In [13]:
D_elas(ε) = d_mat ⋅² ε

D_piezo(∇) = ∇ ⋅ a_mat 

D_piezo (generic function with 1 method)

## FE formulation

In [14]:
function project(q,model,dΩ,order)
  reffe = ReferenceFE(lagrangian,Float64,order)
  V = FESpace(model,reffe,conformity=:L2)
  a(u,v) = ∫( u*v )*dΩ
  l(v) = ∫( v*q )*dΩ #+ ∫( v*h )*dΓ
  op = AffineFEOperator(a,l,V,V)
  qh = Gridap.solve(op)
  qh
end

project (generic function with 1 method)

In [15]:
Γ = BoundaryTriangulation(model,tags=["NeumannTop"])
dΓ = Measure(Γ,degree)
σApp = VectorValue{2,Float64}(0,σ₀)

VectorValue{2, Float64}(0.0, 1000.0)

In [16]:
labels = get_face_labeling(model)
dimension = 2
mat_tags = get_face_tag(labels,dimension);

In [17]:
reffe_Disp = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
        V0_Disp = TestFESpace(model,reffe_Disp;
          conformity=:H1,
          dirichlet_tags=["DirichletBot"],
          dirichlet_masks=[(false,true)])

uh = zero(V0_Disp)

SingleFieldFEFunction():
 num_cells: 29132
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 18081684107655847535

In [18]:
reffe_ElecPot = ReferenceFE(lagrangian,Float64,order)
V0_ElecPot  = TestFESpace(model,reffe_ElecPot;
  conformity=:H1,
  dirichlet_tags=["ElectricPotentialBot","ElectricPotentialTop"],
  dirichlet_masks=[true,true])

UnconstrainedFESpace()

In [19]:
V0 = MultiFieldFESpace([V0_Disp,V0_ElecPot])

MultiFieldFESpace()

In [20]:
 function   stepDispElecPot(uh_in,phih_in,vApp,phiApp)
    
        uApp1(x) = VectorValue(vApp,0.0)
        U_Disp = TrialFESpace(V0_Disp,[uApp1])
    
        phiApp1(x) = phiApp
        phiApp2(x) = 0
        U_ElecPot = TrialFESpace(V0_ElecPot,[phiApp1,phiApp2])
    
        U = MultiFieldFESpace([U_Disp,U_ElecPot])
    
        a((u,ϕ),(v,ψ)) = ∫( (ε(v) ⊙ (σ_elas∘(ε(u)))) + (∇(v) ⊙ (σ_piezo∘(∇(ϕ)))) - (∇(ψ)⋅(D_piezo∘(∇(ϕ)))) + (∇(ψ)⋅(D_elas∘(ε(u)))) )*dΩ
        b((v,ψ)) = ∫( v⋅σApp )*dΓ
    
        op = AffineFEOperator(a,b,U,V0)
        uhPhi = Gridap.solve(op)
        uh_out,phih_out = uhPhi
    
    return uh_out,phih_out
end

stepDispElecPot (generic function with 1 method)

In [21]:
vApp = 0.0
phiApp = 0.0
ϕPrev = CellState(0.0,dΩ)
ϕh = project(ϕPrev,model,dΩ,order)
 
uh,ϕh = stepDispElecPot(uh,ϕh,vApp,phiApp)

(SingleFieldFEFunction(), SingleFieldFEFunction())

In [22]:
 writevtk(Ω,"results_PlateWithHoleA21",cellfields=
        ["uh"=>uh,"phi"=>ϕh, "epsi"=>ε(uh), "ElecF"=>-∇(ϕh), "SigmaE"=>(σ_elas∘(ε(uh)))])

(["results_PlateWithHoleA21.vtu"],)

In [23]:
using Gridap.CellData
using Gridap.Fields
using Gridap.Arrays
using Gridap.Geometry
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.MultiField
using Gridap.Polynomials
using Gridap.Algebra
using Gridap.TensorValues
using Gridap.Helpers
using WriteVTK
using PyPlot

### Extract values of Displacement , Strain and Stress components

In [24]:
Gr = get_grid(model)
GrTop = get_grid_topology(model)
PolyType = get_polytopes(model);

In [25]:
FeSp = get_fe_space(uh)
get_cell_dof_ids(FeSp);

In [26]:
Nₑ = num_cells(model)
elem = get_cell_node_ids(Gr);

In [27]:
uv = get_cell_dof_values(uh)
uv[1];

In [28]:
u1El = [uv[i][1:3] for i in 1:Nₑ]
u2El = [uv[i][4:6] for i in 1:Nₑ];

In [29]:
nodes = get_node_coordinates(Gr)
Nₙ = num_nodes(model)
nodeCoordX = [nodes[i][1] for i in 1:Nₙ]
nodeCoordY = [nodes[i][2] for i in 1:Nₙ];

In [30]:
u1 = zeros(Nₙ,1)
u2 = zeros(Nₙ,1)
for iel in 1:Nₑ
    ElNodeIndx = elem[iel]
    u1[ElNodeIndx,1] = u1El[iel]
    u2[ElNodeIndx,1] = u2El[iel]
end

In [31]:
dataStrain = get_array(ε(uh))
ϵVec = return_value(dataStrain,Point(nodeCoordX[1],nodeCoordY[1]))
ϵ11El = [ϵVec[i][1,1] for i in 1:Nₑ] 
ϵ12El = [ϵVec[i][1,2] for i in 1:Nₑ] 
ϵ22El = [ϵVec[i][1,3] for i in 1:Nₑ];

In [32]:
dataStress = get_data(σ_elas∘(ε(uh)))
σVec = return_value(dataStress,Point(nodeCoordX[1],nodeCoordY[1]))
σ11E = [σVec[i][1,1] for i in 1:Nₑ] 
σ12E = [σVec[i][1,2] for i in 1:Nₑ] 
σ22E = [σVec[i][1,3] for i in 1:Nₑ];

In [33]:
dataStressp = get_data(σ_piezo∘(-∇(ϕh)))
PVec = return_value(dataStressp,Point(nodeCoordX[1],nodeCoordY[1]))
P11El = [PVec[i][1] for i in 1:Nₑ] 
P12El = [PVec[i][2] for i in 1:Nₑ] 
P21El = [PVec[i][3] for i in 1:Nₑ]
P22El = [PVec[i][4] for i in 1:Nₑ];

In [34]:
σ11El = σ11E + P11El
σ12El = σ12E + P12El
σ22El = σ22E + P22El;

In [35]:
dataDisp = get_data(D_elas∘(ε(uh)))
DVec = return_value(dataDisp,Point(nodeCoordX[1],nodeCoordY[1]))
D11E = [DVec[i][1] for i in 1:Nₑ] 
D22E = [DVec[i][2] for i in 1:Nₑ];

In [36]:
dataDisp1 = get_data(D_piezo∘(∇(-ϕh)))
AVec = return_value(dataDisp1,Point(nodeCoordX[1],nodeCoordY[1]))
a11El = [AVec[i][1] for i in 1:Nₑ] 
a22El = [AVec[i][2] for i in 1:Nₑ];

In [37]:
D11El = D11E + a11El
D22El = D22E + a22El;

In [38]:
ϵ11 = zeros(Nₙ,1)
ϵ22 = zeros(Nₙ,1)
ϵ12 = zeros(Nₙ,1)
σ11 = zeros(Nₙ,1)
σ22 = zeros(Nₙ,1)
σ12 = zeros(Nₙ,1)
D11 = zeros(Nₙ,1)
D22 = zeros(Nₙ,1)
for iel in 1:Nₑ
    ElNodeIndx = elem[iel]
    ϵ11[ElNodeIndx,1] = ϵ11El[iel]*ones(3,1)
    ϵ12[ElNodeIndx,1] = ϵ12El[iel]*ones(3,1)
    ϵ22[ElNodeIndx,1] = ϵ22El[iel]*ones(3,1)
    σ11[ElNodeIndx,1] = σ11El[iel]*ones(3,1)
    σ12[ElNodeIndx,1] = σ12El[iel]*ones(3,1)
    σ22[ElNodeIndx,1] = σ22El[iel]*ones(3,1)
    D11[ElNodeIndx,1] = D11El[iel]*ones(3,1)
    D22[ElNodeIndx,1] = D22El[iel]*ones(3,1)
end

In [39]:
cell = MeshCell(VTKCellTypes.VTK_TRIANGLE, elem[1])
for iel in 2:Nₑ
c = MeshCell(VTKCellTypes.VTK_TRIANGLE, elem[iel])
cell = [cell; c]
end
vtk_grid("./ResultsPlateWithHoleCompA2",nodeCoordX,nodeCoordY,cell) do vtk 
vtk["u", VTKPointData()] = u1
vtk["v", VTKPointData()] = u2  
vtk["eps11", VTKPointData()] = ϵ11
vtk["eps22", VTKPointData()] = ϵ22
vtk["eps12", VTKPointData()] = ϵ12
vtk["NormStr11", VTKPointData()] = σ11/σ₀
vtk["NormStr22", VTKPointData()] = σ22/σ₀
vtk["NormStr12", VTKPointData()] = σ12/σ₀
vtk["NormPot11", VTKPointData()] = D11
vtk["NormPot22", VTKPointData()] = D22
end

1-element Vector{String}:
 "./ResultsPlateWithHoleCompA2.vtu"

In [40]:
maximum(σ22)/σ₀

2.7827438100378687